In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

from utils import key2word, key2ruword


fg_file = "../data/in_data/fg_R9_manifest.tsv"
mapping_file = "../data/in_data/fg_R9_ukb_mapping.tsv"
ukb_file = "../data/in_data/panukb_phenotype_manifest.csv"
ukb_neale_file = "../data/in_data/ukb_manifest.csv"

merged_to = "../data/merged_bio.tsv"
codes_to = "../data/codes.tsv"

In [2]:
fg_cols = ['phenocode', 'name', 'category']
mapping_cols = ['name', 'category', 'fg_phenotype', 'ukbb_phenotype']
ukb_cols = ['phenocode', 'description', "description_more"]

In [3]:
# read
fg = pd.read_csv(fg_file, sep='\t')[fg_cols].drop_duplicates(keep='first')
ukb = pd.read_csv(ukb_file, sep=',')[ukb_cols].drop_duplicates(keep='first')
# neale = pd.read_csv(ukb_neale_file, sep=',')
mapping = pd.read_csv(mapping_file, sep='\t')[mapping_cols].drop_duplicates(keep='first')




display(fg.head())
display(ukb.head())
# display(neale.head())
display(mapping.head())
print(fg.shape, ukb.shape, mapping.shape)

,phenocode,name,category
0,AB1_ACTINOMYCOSIS,Actinomycosis,I Certain infectious and parasitic diseases (A...
1,AB1_AMOEBIASIS,Amoebiasis,I Certain infectious and parasitic diseases (A...
2,AB1_ANOGENITAL_HERPES_SIMPLEX,Anogenital herpesviral [herpes simplex] infection,I Certain infectious and parasitic diseases (A...
3,AB1_ASPERGILLOSIS,Aspergillosis,I Certain infectious and parasitic diseases (A...
4,AB1_ATYPICAL_CNS_VIRUS,Atypical virus infections of central nervous s...,I Certain infectious and parasitic diseases (A...


,phenocode,description,description_more
0,30600,Albumin,NaN
1,30610,Alkaline phosphatase,NaN
2,30620,Alanine aminotransferase,NaN
3,30630,Apolipoprotein A,NaN
4,30640,Apolipoprotein B,NaN


,name,category,fg_phenotype,ukbb_phenotype
0,Aspergillosis,I Certain infectious and parasitic diseases (A...,AB1_ASPERGILLOSIS,B44
1,Candidiasis,I Certain infectious and parasitic diseases (A...,AB1_CANDIDIASIS,B37
2,Dermatophytosis,I Certain infectious and parasitic diseases (A...,AB1_DERMATOPHYTOSIS,B35
3,Diarrhoea and gastroenteritis of presumed infe...,I Certain infectious and parasitic diseases (A...,AB1_GASTROENTERITIS_NOS,A09
4,Helminthiases,I Certain infectious and parasitic diseases (A...,AB1_HELMINTIASES,134


(2272, 3) (3925, 3) (679, 4)


Проверим множества

In [4]:
mapping.fg_phenotype.unique().shape, mapping.ukbb_phenotype.unique().shape

((679,), (657,))

Получается буквально несколько юкбишных дублируются. Ок

## Мержим

In [5]:
def merge_columns(df, col1, col2):
    """
    Merge two columns in a DataFrame based on specified rules.
    - If values are equal, pick that one.
    - If one is NaN, pick the other.
    - If they are not equal, concatenate them into a string.
    """
    def merge_row(row):
        if row[col1] == row[col2]:
            return row[col1]
        elif pd.isna(row[col1]):
            return row[col2]
        elif pd.isna(row[col2]):
            return row[col1]
        else:
            return str(row[col1]) + " " + str(row[col2])

    return df.apply(merge_row, axis=1)


In [6]:
merged = ukb.add_suffix('_ukb').merge(
    mapping.add_suffix('_meta'), 
    how='outer', 
    left_on='phenocode_ukb', 
    right_on='ukbb_phenotype_meta',
).rename(columns={"fg_phenotype_meta":"fg_phenotype", "ukbb_phenotype_meta": "ukbb_phenotype"}).merge(
    fg.add_suffix('_fg'), 
    how='outer', 
    left_on='fg_phenotype', 
    right_on='phenocode_fg',
)

merged['ukb_code'] = merge_columns(merged, 'phenocode_ukb', 'ukbb_phenotype')
merged['fg_code'] = merge_columns(merged, 'fg_phenotype', 'phenocode_fg')
merged['fg_name'] = merge_columns(merged, 'name_meta',  'name_fg')

merged = merged[['ukb_code', 'fg_code', 'fg_name', 'category_fg', 'category_meta', 'description_ukb', 'description_more_ukb']]
merged.sample(5)

,ukb_code,fg_code,fg_name,category_fg,category_meta,description_ukb,description_more_ukb
99,2000,NaN,NaN,NaN,NaN,Worry too long after embarrassment,"ACE touchscreen question ""Do you worry too lon..."
5222,NaN,M13_HALLUDEFORMOTH,Other deformity of hallux (acquired),XIII Diseases of the musculoskeletal system an...,NaN,NaN,NaN
2380,495.2,NaN,NaN,NaN,NaN,Asthma with exacerbation,NaN
2180,348.8,NaN,NaN,NaN,NaN,"Encephalopathy, not elsewhere classified",NaN
3602,O62,O15_LABOUR_ABNORM_FORCES,Abnormalities of forces of labour,"XV Pregnancy, childbirth and the puerperium (O...","XV Pregnancy, childbirth and the puerperium (O...",O62 Abnormalities of forces of labour,truncated: true


Все, теперь сохраним, так как будем этим пользоваться. 

In [7]:
merged.to_csv(merged_to, sep='\t', index=False)

## Коды нашего биобанка

In [8]:
ru_en_names = pd.DataFrame([key2word, key2ruword]).T
ru_en_names.columns = ['en', 'ru']
ru_en_names = ru_en_names.reset_index().rename(columns={'index':'codes'})
ru_en_names

,codes,en,ru
0,type_днк,DNA,ДНК
1,type_инг,Whole blood in RNA stabilizer,Цельная кровь в консерванте для для стабилизац...
2,type_моча,Urine,Моча
3,type_пл,Plasma,Плазма
4,type_пл(фф),Plasma (fetal fraction),Плазма (фетальная фракция)
5,type_плац,Placenta,Плацента
6,type_пуп_инг,Cord blood in RNA stabilizer,Пуповинная кровь в консерванте для стабилизаци...
7,type_пуп_лей,Cord blood buffy coat,Лейкоцитарная пленка из пуповинной крови
8,type_пуп_пл,Cord blood plasma,Плазма пуповинной крови
9,type_пуп_сыв,Cord blood serum,Сыворотка пуповинной крови


In [9]:
ru_en_names.to_csv(codes_to, sep='\t', index=False)